In [2]:
import os.path
import shutil
import pandas as pd



def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)
    else:
        shutil.rmtree(path)
        os.makedirs(path)

ds = pd.read_csv('passes.csv')

# removing coordinates columns
columns = []
for i in range(1, 29):
    columns.append( "x_" + str(i))
    columns.append( "y_" + str(i))
ds.drop(columns, axis=1, inplace=True)

# sort by time for easy search for specific match
ds.sort_values(by=['time_start'], inplace=True)


matches = []

# extract csv for each match
for i in range(14):
    print("Extracting Match ",  i)

    # extract the first row
    matches.append(pd.DataFrame(columns=ds.columns))
    row = ds.iloc[0]
    matches[i] = pd.concat([matches[i], row.to_frame().T], ignore_index=True)
    ds.drop(row.name, inplace=True)

    # extract the rest of the rows
    while True:
        row = ds.query(f"sender_id == {row['receiver_id']}  and time_start >= {row['time_end']}")

        # if no more rows found following the previous row
        if row.empty:
           break
        else:
            row = row.iloc[0]
            matches[i] = pd.concat([matches[i], row.to_frame().T], ignore_index=True)
        # remove the row from the original dataset after extracting it
        ds.drop(row.name, inplace=True)

# save the matches as csv
create_dir("matches_csv")
for i in range(14):
    matches[i].to_csv(f"matches_csv/match_{i+1}.csv", index=False)


# window width 4 min for now
# 14 matches
# 13 snapshots
# in milli
window_size = 4 * 60 * 1000
# n nodes
snapshot = "14\n"
# id assign
for i in range(14):
    snapshot += f"{i} \"{"A" + str(21+i)}\""
    snapshot += "\n"

snapshots = [[snapshot for _ in range(13)] for _ in range(14) ]
# edges
for ith_game in range(14): # matches
    # iterate for all matches
    for ith_snapshot in range(13):
        # iterate for all snapshots [13 in total] means 13 text file in each match folder
        for ith_node in range(14): # nodes
            # iterate for all the nodes in a particular snapshot
            matchs = matches[ith_game].query(f"sender_id == {ith_node+1} and time_start > {ith_snapshot*window_size} and time_start <= {(ith_snapshot+1)*window_size} and receiver_id <= 14")
            edge_size = matchs.shape[0]
            snapshots[ith_game][ith_snapshot] += f"{edge_size}\n"
            for ith_edge in range(edge_size):
                snapshots[ith_game][ith_snapshot] += f"{ith_node} {matchs.iloc[ith_edge]['receiver_id']-1}\n"

# save the snapshots
create_dir("target-graphs")

for i in range(14):
    os.makedirs(f"target-graphs/match_{i+1}")
    for j in range(13):
        with open(f"target-graphs/match_{i+1}/dg{j+1}.txt", "w") as f:
            f.write(snapshots[i][j])

Extracting Match  0
Extracting Match  1
Extracting Match  2
Extracting Match  3
Extracting Match  4
Extracting Match  5
Extracting Match  6
Extracting Match  7
Extracting Match  8
Extracting Match  9
Extracting Match  10
Extracting Match  11
Extracting Match  12
Extracting Match  13


In [3]:
 # window width 4 min for now
# 14 matches
# 13 snapshots
# in milli
window_size = 4 * 60 * 1000
# n nodes
snapshot = "14\n"
# id assign
for i in range(14):
    snapshot += f"{i} \"{"A" + str(41+i)}\""
    snapshot += "\n"

snapshots = [[snapshot for _ in range(13)] for _ in range(14) ]
# edges
for ith_game in range(14): # matches
    # iterate for all matches
    for ith_snapshot in range(13):
        # iterate for all snapshots [13 in total] means 13 text file in each match folder
        for ith_node in range(14): # nodes
            # iterate for all the nodes in a particular snapshot
            matchs = matches[ith_game].query(f"sender_id == {ith_node+15} and time_start > {ith_snapshot*window_size} and time_start <= {(ith_snapshot+1)*window_size} and receiver_id > 14")
            edge_size = matchs.shape[0]
            snapshots[ith_game][ith_snapshot] += f"{edge_size}\n"
            for ith_edge in range(edge_size):
                snapshots[ith_game][ith_snapshot] += f"{ith_node} {matchs.iloc[ith_edge]['receiver_id']-15}\n"

# save the snapshots
create_dir("target-graphs-away")

for i in range(14):
    os.makedirs(f"target-graphs-away/target-graphs/match_{i+1}")
    for j in range(13):
        with open(f"target-graphs-away/target-graphs/match_{i+1}/dg{j+1}.txt", "w") as f:
            f.write(snapshots[i][j])